## 3. Fine-tuning

고유의 데이터셋을 이용해서 기존의 사전 학습된 모델을 fine-tuning하기

학습 목적

1. 허브에서 대규모 데이터셋 가져오기
2. 고급 Trainer API 사용해서 모델 미세 조정하기
3. 사용자 지정 학습 루프 사용
4. Accelerate 라이브러리 활용 -> 분산 환경에서 사용자 지정 학습 루프 쉽게 실행

### 1. 데이터 처리 작업

단일 배치 기반으로 시퀀스 분류기를 학습하기

In [3]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

ckpt = "bert-base-uncased"
tokenizer= AutoTokenizer.from_pretrained(ckpt)
model = AutoModelForSequenceClassification.from_pretrained(ckpt)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

batch = tokenizer(sequences,
                  padding=True,
                  truncation=True,
                  return_tensors="pt")

batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 허브에서 데이터셋 로딩

허브에는 다양한 언어로 구축된 여러 데이터셋들이 존재

`Datasets` 라이브러리는 허브에서 데이터셋을 다운로드하고 캐시 기능을 수행하는 쉬운 명령어를 제공한다

In [6]:
# MRPC 데이터셋 다운로드
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [8]:
# DatasetDict 는 파이선의 딕셔너리와 같이 키값으로 raw_datasets 개체의 개별 집합에 접근 가능
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [9]:
# 어떤 정수가 어떤 레이블에 해당하는지 파악하기 : raw_train_datase의 feature 보기
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

### 데이터셋 전처리

텍스트를 모델이 이해할 수 있는 숫자로 변환해야 한다(토크나이저가 담당)

토크나이저에 단일 문장/다중 문장 리스트 입력이 가능하므로, 각 쌍의 모든 첫 번째 문장과 두 번째 문장을 직접 토큰화할 수 있다

In [10]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

두 시퀀스를 모델에 바로 전달하여 예측을 얻을 수는 없다.

두 시퀀스를 쌍으로 처리하고 적절한 전처리를 적용해야 한다.

In [11]:
inputs = tokenizer("This is the first sentence", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
# 다시 디코딩
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

[CLS] 문장1 [SEP] : `token_type_id` 0

문장2 [SEP] : `token_type_id` 1

모델이 `token_type_id` 형태의 입력 형식으로 학습을 진행하였기 때문에 `tokenizer` 가 `token_type_id` 를 반환

학습 데이터셋을 전처리하는 방법

```python
tokenized_dataset = tokenizer(
    raw_datasets['train']['sentence1'],
    raw_datasets['train']['sentence2'],
    padding=True,
    truncation=True,
)
```

위의 방법은 별도의 파이썬 딕셔너리를 반환한다.\
또한, 이 방법은 토큰화하는 동안 전체 데이터셋을 저장할 충분한 공간의 RAM이 있는 경우에만 작동한다.

특정 데이터를 `dataset` 객체로 유지하기 위해 `Dataset.map()` 메소드를 사용한다.\
`map()` 은 개별 데이터에 함수를 적용하여 작동한다. 이를 이용해 입력을 토큰화하는 함수를 정의할 수 있다.

```python
def tokenize_function(example):
    return tokenizer(
        example["sentence1"], example["sentence2"],
        truncation=True
    )
```

위와 같은 함수를 이용해서 `map()` 호출에서 `bathced=True` 옵션을 사용하면 토큰화 속도가 빨라진다.

In [13]:
def tokenize_function(example):
    return tokenizer(
        example["sentence1"], example["sentence2"],
        truncation=True
    )

In [14]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

`datasets` 은 데이터셋에 새로운 필드들을 추가한다.\
이 필드들은 전처리 함수에서 반환된 사전의 각 키에 해당된다. (`input_ids`, `token_type_ids`, `attention_mask`)

`num_proc` 매개변수를 전달하여 `map()` 으로 전처리 기능을 적용할 때 다중 처리를 사용할 수도 있다.

### 동적 패딩

마지막으로 전체 요소들을 배치로 분리할 때 가장 긴 요소의 길이로 모든 예제를 채운다. 이를 동적 패딩이라고 한다.

__collate function__ : 샘플들을 함께 모아서 지정된 크기의 배치로 구성하는 역할을 하는 함수\
`DataLoader` 를 빌들할 때 전달할 수 있는 함수

예) `DataCollatorWithPadding` : 토크나이저를 입력으로 받는다

In [21]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
# 예시: 샘플 가져오기
samples = tokenized_datasets['train'][:8]
samples = {k: v for k, v in samples.items() if k not in ['idx', 'sentence1','sentence2']}
[len(x) for x in samples['input_ids']]

[50, 59, 47, 67, 59, 50, 62, 32]

In [23]:
# 동적 패딩은 이 경우 배치 내부의 최대 길이인 67로 패딩된다
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}